## This script will extract data off avito for further analysis

In [133]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import clear_output

In [134]:
#this functions takes an avito car page and extracts data from it
def getInternalData(link):
    data={}
    page=requests.get(link)
    print(link)
    soup = BeautifulSoup(page.content,"html.parser")
    annonceText=soup.find("p",{"style":"word-wrap: break-word;"}).text
    dataPanel=soup.find("aside",{"class":"panel panel-body panel-info"})
    dataList=dataPanel.find("ul")
    for li in dataList.find_all("li"):
        if li.find("strong")!=None:
            title=li.find("strong").text
            
            if li.find("a")==None:
                
                value=li.find("h2").text.split("\n")[1]
            else:
                value=li.find("a").text
            data[title]=value
        else:
            pass
       
    return data
    
    

In [135]:
#setup how many pages you want using pageDeepness
pageDeepness=2



totalItems=pageDeepness*35 
currentPage=1
data=[]
for i in range(1,pageDeepness+1):
    clear_output()
    print("--------------Currently processing page "+str(i)+" -------------------------------------------------------")
    
    
    startItem=(i-1)*35 +1
    endItem=startItem+35
    page=requests.get("https://www.avito.ma/fr/maroc/voitures-à_vendre?o="+str(i))
    soup = BeautifulSoup(page.content,"html.parser")
    for j in range(startItem,endItem):
        idItem="li-item-"+str(j)+" "
        item=soup.find("div", {"id": idItem})
        name=item.find("h2").find("a").text
        price=item.find("span",{"class":"price_value"}).text
        city=item.find("span",{"class":"item-info-extra fs14"}).find("a").text
        proOrNot=False if item.find("span",{"class":"item-info-extra fs14"}).find("strong")==None else True
        link=item.find("h2").find("a",href=True)['href']
        dataPoint=getInternalData(link)
        dataPoint["name"]=name
        dataPoint["city"]=city
        dataPoint["proOrNot"]=proOrNot
        data.append(dataPoint)
        print("progress: "+str(j/totalItems)+"%")

clear_output()       
print("All Done :3")
        
    


    

    
    

All Done :3


In [111]:
#put it into a pandas df
dataframe=pd.DataFrame(data)
dataframe.head(5)

,Année-Modèle:,Kilométrage:,Marque:,Modèle:,Puissance fiscale:,Secteur:,Type de carburant:,Type:,city,name,proOrNot
0,2009,200 000 - 249 999,Peugeot,308,6 CV,NaN,Diesel,"Voitures, Offre",Casablanca,Peugeot 308 -2009,False
1,2017,35 000 - 39 999,Audi,RS3,11 CV,maarif,Essence,"Voitures, Offre",Casablanca,Audi RS3 -2017,True
2,2014,0 - 4 999,Hyundai,Santa Fe,5 CV,centre,Diesel,"Voitures, Offre",El Jadida,Hyundai Santa Fe Diesel -2014,True
3,2015,100 000 - 109 999,Hyundai,i 10,5 CV,hay_mohammedi,Essence,"Voitures, Offre",Fès,Hyundai i 10 Essence tt -2015,True
4,2003,170 000 - 179 999,Volkswagen,GOLF 4,8 CV,NaN,Diesel,"Voitures, Offre",Agadir,golf4 -2003,False


## Export it to excel

In [116]:
dataframe.to_excel("avito_data.xlsx")

## TO DO :
* Clean the data
* Make a model to predict price
